<a href="https://colab.research.google.com/github/masoudem/solar-power-forecasting-with-LSTM/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#مشخص کردن میزان سخت افزار
!nvidia-smi

Sat Sep 25 18:43:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Different methods for predicting output power

#### Python libraris, as follows:

In [ ]:
#کتابخانه های مورد نیاز برای اجرای کدها
import pandas as pd
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
from datetime import datetime
import math
from math import sqrt
from matplotlib import pyplot
import matplotlib.pyplot as plt
py.init_notebook_mode(connected=True)
%matplotlib inline
import numpy
import numpy as np
from numpy import mean, std, array, argmax, concatenate
from keras.models import Sequential, load_model
from keras.layers import Activation, Bidirectional,BatchNormalization 
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Bidirectional
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression

#### Read data

In [ ]:
#آپلود کردن داده‌ها از کامپیوتر شخصی
#uplode file pv_01
import io
import os
from google.colab import drive
from google.colab import files
uploaded = files.upload()

In [ ]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
data = pd.read_csv(io.StringIO(uploaded['pv_01.csv'].decode('utf-8')),  parse_dates=['time'], index_col = 'time')

#### Data vision

In [ ]:
data.info()

In [ ]:
print(data.head())
print(data.tail())
print(data.shape)
print(data.dtypes)

In [ ]:
data.describe()

#### Feature selection

In [ ]:
data.hist(bins=50, figsize=(15,10))
plt.show()

In [ ]:
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(10,10))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

#### Data processing

In [ ]:
#آماده سازی اطلاعات
numpy.random.seed(7)
df = data.iloc[:, 0:].values
scaler = MinMaxScaler(feature_range = (0, 1))
pr_df = scaler.fit_transform(df)
data = np.array(pr_df)
data.shape

#### Data spilit

In [ ]:
X = []
Y = []
for i in range(20, 7300):
  X.append(data[i-20:i, 0: ])
  Y.append(data[i, 0])
X, Y = np.array(X), np.array(Y)
print(X.shape)
print(Y.shape)

### RMSE

In [ ]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

### RAdam

In [ ]:
#کدهای مربوط به بهینه سازی
from keras.legacy import interfaces
from keras.optimizers import Optimizer
import keras.backend as K


class RAdam(Optimizer):
    """RAdam optimizer.
    Default parameters follow those provided in the original Adam paper.
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
    # References
        - [RAdam - A Method for Stochastic Optimization]
          (https://arxiv.org/abs/1908.03265)
        - [On The Variance Of The Adaptive Learning Rate And Beyond]
          (https://arxiv.org/abs/1908.03265)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., **kwargs):
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)
        rho = 2 / (1 - self.beta_2) - 1
        rho_t = rho - 2 * t * beta_2_t / (1 - beta_2_t)
        r_t = K.sqrt(
            K.relu(rho_t - 4) * K.relu(rho_t - 2) * rho / ((rho - 4) * (rho - 2) * rho_t)
        )
        flag = K.cast(rho_t > 4, K.floatx())

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            mhat_t = m_t / (1 - beta_1_t)
            vhat_t = K.sqrt(v_t / (1 - beta_2_t))
            p_t = p - lr * mhat_t * (flag * r_t / (vhat_t + self.epsilon) + (1 - flag))

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon}
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### LSTM

In [ ]:
#مدل LSTM
def fit_model_1(X, Y):
  
  # define model
  model = Sequential()
  model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
  model.add(Activation('relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss= root_mean_squared_error, metrics=['mae', 'mse'])


  
  history = model.fit(X, Y, epochs = 100, batch_size = 16, validation_split=0.2)
  return model, history

### Training:

In [ ]:
#آموزش مدلها
model_1, history_1 = fit_model_1(X, Y)
model_1.summary()

In [ ]:
#مشخص کردن overfitting
pyplot.plot(history_1.history['mean_absolute_error'], label='train')
pyplot.plot(history_1.history['val_mean_absolute_error'], label='val')
pyplot.legend()
plt.xlabel('EPOCH')  
plt.ylabel('MAE')
pyplot.show()
pyplot.plot(history_1.history['loss'], label='train')
pyplot.plot(history_1.history['val_loss'], label='val')
pyplot.legend()
plt.xlabel('EPOCH')  
plt.ylabel('RMSE')
pyplot.show()

### Read train data

In [ ]:
#آپلود داده‌های آموزش
#uplode file pv_01
uploaded1 = files.upload()

In [ ]:
for fn in uploaded1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded1[fn])))

In [ ]:
# آماده سازی داده‌ها برای پیشبینی
pvt = pd.read_csv(io.StringIO(uploaded1['pv_01.csv'].decode('utf-8')))
dft = pvt.iloc[:, 1:].values
test_inputs = scaler.fit_transform(dft)
test_inputs = np.array(test_inputs)
print(test_inputs.shape)
test_features = []
for i in range(20, 7280):
  test_features.append(test_inputs[i-20:i, 0: ])
test_features = np.array(test_features)
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], test_features.shape[2]))
print(test_features.shape) 

### Prediction

In [ ]:
#پیشبینی مدلها
predictions_1 = model_1.predict(test_features, verbose=1) 
z = scaler.fit_transform(pvt['power (W)'].values.reshape(-1,1))

### test

In [ ]:
#آماده سازی داده های مقایسه
h = pvt.iloc[20:7280, 1:2].values
h = h.reshape(-1,1)
h.shape

In [ ]:
#نمودار مربوط به رگرسیون
reg = LinearRegression().fit(h, predictions)
r_sq = reg.score(h, predictions)
plt.scatter(h, predictions,edgecolors=(0, 0, 0))
plt.plot(h, reg.predict(h),'k--')
plt.show()
print('coefficient of determination:', r_sq)
print('intercept:', reg.intercept_)
print('slope:', reg.coef_)

In [ ]:
#نمودار مقایسه ای توان واقعی و توان پیش بینی
plt.figure(figsize=(18,6))  
plt.plot(h[100:200] , color='blue', label='Actual Power')  
plt.plot(predictions[100:200] , color='red', label='Predicted Power')  
plt.title('Power Prediction')  
plt.xlabel('Time')  
plt.ylabel('Power(W)')  
plt.legend()
plt.show()

In [ ]:
#محاسبه خطای پیشبینی
rmse = sqrt(mean_squared_error(h, predictions))
print('Test RMSE: %.6f' % rmse)
mse = mean_squared_error(h, predictions)
print('Test MSE: %.6f' % mse)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(h, predictions)
print('Test MAE: %.6f' % mae)
from sklearn.metrics import r2_score
r2 = r2_score(h, predictions)
print('Test R^2: %.6f' % r2)
from sklearn.metrics import explained_variance_score
variance = explained_variance_score(h, predictions)
print('Test Variance: %6f' % variance)